<a href="https://colab.research.google.com/github/deftydev/Crew_Agents/blob/main/RAG_FAISS_LANGCHAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu  tiktoken requests

In [2]:
from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from typing import List, Dict, Any
import requests
import numpy as np
import faiss
import os
import getpass

In [3]:
!pip install -q langchain_google_genai

In [4]:
!pip install sentence-transformers

In [5]:
LANGSMITH_TRACING= True
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_PROJECT="chat_with_langchain"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [6]:
!pip install pypdf

In [7]:
!pip install -U langchain langchain-core langchain-community

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

pdf_path = "/content/Devansh_resume.docx.pdf"

loader = PyPDFLoader(pdf_path)

documents = loader.load()  ##Use unstructured PDF Loader if you want to load images, table , figures and text

In [9]:
documents

[Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Devansh_resume.docx', 'source': '/content/Devansh_resume.docx.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Devansh  Gupta  \n📍\n \nNoida,\n \nIndia\n \n|\n \n📧\n \ndevansh.chatgpt@gmail.com\n \n|\n \n📞\n \n+91\n \n7983449648\n 🔗  linkedin.com/in/devansh-gupta-145077189|  🖥  https://github.com/deftydev  \n \nPROFESSIONAL  SUMMARY  \nResults-driven  Machine  Learning  Engineer  with  3+  years  of  experience  specializing  in  Computer  \nVision\n \nand\n \nGenerative\n \nAI.\n \nProficient\n \nin\n \nPython,\n \nSQL,\n \nDeep\n \nLearning,\n \nand\n \nAI-powered\n \nsearch\n \nengines.Expertise\n \nin\n \nRAG\n \n(Retrieval-Augmented\n \nGeneration),\n \nmodel\n \noptimization,\n \nprompt\n \nengineering,\n \nand\n \nAI-powered\n \nsearch\n \nengines\n.\n \nProven\n \nability\n \nto\n \nintegrate\n \nNLP,\n \ndeep\n \nlearning,\n \nand\n \nvect

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # Max tokens/characters per chunk
    chunk_overlap=50     # Overlap between chunks to preserve context
)

split_docs = text_splitter.split_documents(documents)

In [11]:
split_docs

[Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Devansh_resume.docx', 'source': '/content/Devansh_resume.docx.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Devansh  Gupta  \n📍\n \nNoida,\n \nIndia\n \n|\n \n📧\n \ndevansh.chatgpt@gmail.com\n \n|\n \n📞\n \n+91\n \n7983449648\n 🔗  linkedin.com/in/devansh-gupta-145077189|  🖥  https://github.com/deftydev'),
 Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Devansh_resume.docx', 'source': '/content/Devansh_resume.docx.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='PROFESSIONAL  SUMMARY  \nResults-driven  Machine  Learning  Engineer  with  3+  years  of  experience  specializing  in  Computer  \nVision\n \nand\n \nGenerative\n \nAI.\n \nProficient\n \nin\n \nPython,\n \nSQL,'),
 Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '

In [12]:
print(f"Number of chunks: {len(split_docs)}")
print("\nSample chunk:\n", split_docs[0].page_content)

Number of chunks: 29

Sample chunk:
 Devansh  Gupta  
📍
 
Noida,
 
India
 
|
 
📧
 
devansh.chatgpt@gmail.com
 
|
 
📞
 
+91
 
7983449648
 🔗  linkedin.com/in/devansh-gupta-145077189|  🖥  https://github.com/deftydev


In [13]:
pip install -U langchain-huggingface

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.in_memory import InMemoryDocstore

In [15]:
# embedding_model.embed_documents(["This is a test", "Second sentence"])

In [16]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

sample_embedding = embedding_model.embed_query("test")
dimension = len(sample_embedding)

print(dimension)

faiss_index = faiss.IndexFlatL2(dimension)

vector_store = FAISS(
    embedding_function=embedding_model,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(split_docs)
vector_store.save_local("faiss_index")

384


In [17]:
vector_store = FAISS.load_local("faiss_index", embeddings=embedding_model , allow_dangerous_deserialization=True)

In [18]:
query = "Explain about devansh."

# Retrieve top 3 relevant chunks
retrieved_docs = vector_store.similarity_search(query, k=3)

# Print the retrieved chunks
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n--- Result {i} ---\n")
    print(doc.page_content)


--- Result 1 ---

Devansh  Gupta  
📍
 
Noida,
 
India
 
|
 
📧
 
devansh.chatgpt@gmail.com
 
|
 
📞
 
+91
 
7983449648
 🔗  linkedin.com/in/devansh-gupta-145077189|  🖥  https://github.com/deftydev

--- Result 2 ---

PROFESSIONAL  SUMMARY  
Results-driven  Machine  Learning  Engineer  with  3+  years  of  experience  specializing  in  Computer  
Vision
 
and
 
Generative
 
AI.
 
Proficient
 
in
 
Python,
 
SQL,

--- Result 3 ---

EDUCATION
 
Bachelor  of  Technology  (B.Tech)  
JSSATE
 
,
 
Noida
 
|
 
2017
 
-
 
2021


In [19]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


In [20]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI  # For Gemini

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

prompt = ChatPromptTemplate.from_template(
    "Answer the question based on the context below:\n\n{context}\n\nQuestion: {question}"
)

rag_chain = (
    {"context": lambda x: vector_store.similarity_search(x["question"], k=3), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke({"question": "What is YOLO object detection?"})
print(response)

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions 